In [ ]:
from tqdm import tqdm
import os
import numpy as np
import pandas as pd
from ogb.lsc import PCQM4Mv2Dataset,PygPCQM4Mv2Dataset
import ogb
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import matplotlib.pyplot as plt
from torch_geometric.utils.smiles import to_smiles, from_smiles

In [ ]:
print(rdkit.__version__) #2021.03.5
print(ogb.__version__) #1.3.3

In [ ]:
suppl = Chem.SDMolSupplier('../../data/pcqm4m-v2-train.sdf')
dataset = PCQM4Mv2Dataset(root = '../../data', only_smiles = True)
data_pyg = PygPCQM4Mv2Dataset(root='../../data')

In [ ]:
type(suppl[0])

In [ ]:
print(dataset[0])
mol = suppl[0]
mol=Chem.RemoveHs(mol) # removes hydrogen from molecule
Chem.Kekulize(mol, clearAromaticFlags=True)
smile = Chem.MolToSmiles(mol,isomericSmiles=False, kekuleSmiles=True, canonical=True)
print(smile) # same?

In [ ]:
print(to_smiles(from_smiles(smile,False,True)))

#to_smiles(data_pyg[0])
print(from_smiles(smile,False,True))
print(data_pyg[0].edge_index)

In [ ]:
for bondi, bond in enumerate(mol.GetBonds()):
    print(bond.GetIdx())
    print([bondi, bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
# 9 = 1
# 2 = 0

In [ ]:
# Value of the chemical property for each molecule of the dataset
prop_values=[]
for dat in dataset:
    prop_values.append(dat[1])
prop_values_arr=np.array(prop_values)
prop_values_arrt

In [ ]:
plt.hist(prop_values_arr)

In [ ]:
# We get all the smiles from the dataset, and those who do not get "smiled" are in the "fail" list
atomlist=[]
smilesall=[] # Simplified Molecular Input Line Entry Specification 
moliall=[]
fail=[]
isomericSmiles=False # chirality not considered
kekuleSmiles=True
for moli, mol in enumerate(tqdm(suppl)):   
    mol=suppl[moli]
    mol=Chem.RemoveHs(mol) # removes hydrogen from molecule
    Chem.Kekulize(mol, clearAromaticFlags=True)
    try: # trying to transform molecule to SMILE (easier for computer usage format)
        smile = Chem.MolToSmiles(mol,isomericSmiles=isomericSmiles, kekuleSmiles=kekuleSmiles, canonical=True)
        smilesall.append(smile)
        moliall.append(moli)
    except:
        fail.append(moli)
    for atom in mol.GetAtoms():
        atomidx=atom.GetAtomicNum() # nombre de protons dans l'atome
        if atomidx not in atomlist:
            atomlist.append(atomidx)
smilesall2=np.array(smilesall)
print(smilesall2)

In [ ]:
# Refactoring in dictionnary
smilesdict={}
for sidx, s in enumerate(smilesall):
    smilesdict[s]=sidx

In [ ]:
# equivalent to decision tree in the paper
results_all=[]
verbose=False
generatedsmilelist=[]
singlebond = list(Chem.MolFromSmiles("CC").GetBonds())[0]
for molidx in range(len(suppl)):
    if molidx%10000==0:
        print(molidx, end=', ')
    mol=suppl[molidx]
    Chem.Kekulize(mol, clearAromaticFlags=True)
    if mol:
        results_arr=[]
        canrm=[]
        hetero=[]
        for atomi, atom in enumerate(mol.GetAtoms()):
            numnb=len(atom.GetNeighbors())
            if numnb==1 and atom.GetAtomicNum()==6:
                canrm.append(atomi)
            if atom.GetAtomicNum()!=6:
                hetero.append(atomi)
          #print(atomi, numnb)
        nonsingle=[]
        inring=[]
        for bondi, bond in enumerate(mol.GetBonds()):
            bondtyp=bond.GetBondType()
            if bondtyp!=Chem.BondType.SINGLE:
                nonsingle.append(bondi)
            if bond.IsInRing() and bondtyp==Chem.BondType.SINGLE:
                inring.append([bondi, bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
        # remove atom if carbon end atom
        for rmidx in canrm:
            molcopy= Chem.RWMol(mol)
            molcopy.RemoveAtom(rmidx)
            generatedsmile=Chem.MolToSmiles(molcopy,isomericSmiles=isomericSmiles, kekuleSmiles=kekuleSmiles)
            if generatedsmile in smilesdict:
                match=smilesdict[generatedsmile]
                if verbose:
                    print(molidx,'r', rmidx, ':', match)
                results_arr.append([molidx, 'r', rmidx, match])
        # change atom to C if heteroatom
        for cidx in hetero:
            molcopy= Chem.RWMol(mol)
            (molcopy.GetAtoms()[cidx]).SetAtomicNum(6)
            try:
                generatedsmile=Chem.MolToSmiles(molcopy,isomericSmiles=isomericSmiles, kekuleSmiles=kekuleSmiles)
                if generatedsmile in smilesdict:
                    match=smilesdict[generatedsmile]
                    if verbose:
                        print(molidx,'c', cidx, ':', match)
                    results_arr.append([molidx,'c', cidx, match])
            except:
                match=0
        # saturate bond
        for bidx in nonsingle:
            molcopy= Chem.RWMol(mol)
            molcopy.ReplaceBond(bidx, singlebond, preserveProps=False)
            try:
                generatedsmile=Chem.MolToSmiles(molcopy,isomericSmiles=isomericSmiles, kekuleSmiles=kekuleSmiles)
                if generatedsmile in smilesdict:
                    match=smilesdict[generatedsmile]
                    if verbose:
                        print(molidx,'b', bidx, ':', match)
                    results_arr.append([molidx,'b', bidx, match])
            except:
                match=0
        # break ring bond if saturated
        for didx in inring:
            molcopy= Chem.RWMol(mol)
            molcopy.RemoveBond(didx[1],didx[2])
            try:
                generatedsmile=Chem.MolToSmiles(molcopy,isomericSmiles=isomericSmiles, kekuleSmiles=kekuleSmiles)
                if generatedsmile in smilesdict:
                    match=smilesdict[generatedsmile]
                    if verbose:
                        print(molidx,'d', didx[0], ':', match)
                    results_arr.append([molidx,'d', didx[0], match])
            except:
                match=0
        if results_arr!=[]:
            results_all.append(np.array(results_arr))
            
            
results_all2=np.vstack(results_all)

In [ ]:
# split atomwise, bondwise
idx_first=results_all2[:,0].astype('int')
operatoridx=results_all2[:,1]
operatoridx[operatoridx=='c']=0
operatoridx[operatoridx=='r']=1
operatoridx[operatoridx=='b']=2
operatoridx[operatoridx=='d']=3
operatoridx=operatoridx.astype('int')
atombondidx=results_all2[:,2].astype('int')
idx_second=results_all2[:,3].astype('int')
atomwise_idx=np.argwhere(operatoridx<2)[:,0]
bondwise_idx=np.argwhere(operatoridx>=2)[:,0]

In [ ]:
# get explanation values for pairs
explain_val=prop_values_arr[idx_first]-prop_values_arr[idx_second]
results_modif=np.vstack((idx_first, operatoridx,atombondidx,idx_second,explain_val)).T

In [ ]:
# Atomwise
df = pd.DataFrame(results_modif[atomwise_idx,:])
list_columns=['molecule index', 'operation index', 'atom index', 'paired molecule index', 'explanation value']

df.columns =list_columns
for key in list_columns[:-1]:
    print(key)
    tmp=df[key].values.astype(int)
    df[key] = tmp

key=list_columns[-1]
tmp=df[key].values
df[key] = np.round(tmp, 10)

In [ ]:
# Bondwise
df2 = pd.DataFrame(results_modif[bondwise_idx,:])
list_columns=['molecule index', 'operation index', 'bond index', 'paired molecule index', 'explanation value']

df2.columns =list_columns
for key in list_columns[:-1]:
    print(key)
    tmp=df2[key].values.astype(int)
    df2[key] = tmp

key=list_columns[-1]
tmp=df2[key].values
df2[key] = np.round(tmp, 10)